# Day 13: Distress Signal
You'll need to re-order a list of received packets (your puzzle input) to decode the message.

For further instructions, see https://adventofcode.com/2022/day/13

## Import the data

In [1]:
with open('data/packets.txt', 'r') as f:
    puzzle = f.read()

Or, use example input.

In [2]:
# example input
puzzle='''[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]'''

## Reading the input

In [3]:
pairs = puzzle.split('\n\n')
left0 = pairs[0].split('\n')[0]
left0

'[1,1,3,1,1]'

We need to decode these strings as python lists. Our friend Json is good at that, let's bring him in.

In [4]:
import json
left0 = json.loads(left0)
left0

[1, 1, 3, 1, 1]

## Strategy

Packets can contain a mix of integers and lists. We use recursion to navigate the packets and compare values. The first step is to check the datatype of each value and convert integers to lists before comparing them against lists.

In [5]:
def check_order(left, right):
    if type(left) is int and type(right) is list:
        return check_order([left], right)
    elif type(left) is list and type(right) is int:
        return check_order(left, [right])
    elif type(left) is int and type(right) is int:
        if left < right:
            return 'in order'
        elif left > right:
            return 'not in order'
        else:
            return 'keep going'
    elif left == [] and right != []:
        return 'in order'
    elif left != [] and right == []:
        return 'not in order'
    elif left == [] and right == []:
        return 'keep going'
    else: 
        left_value = left.pop(0)
        right_value = right.pop(0)
        order = check_order(left_value, right_value)
        if order == 'keep going':
            return check_order(left, right)
        else:
            return order
        

In [6]:
answer=0
index=0

pairs = puzzle.split('\n\n')
for pair in pairs:
    index += 1
    left = json.loads(pair.split('\n')[0])
    right = json.loads(pair.split('\n')[1])  
    order = check_order(left, right)
    if order == 'in order':
        answer += index
    
answer

13

## Part 2
Ignore blank lines, include two additional divider packets, and put all packets in order. Calculate the decoder key by multiplying the indices of the divider packets.

In [7]:
puzzle = puzzle + '\n\n[[2]]\n[[6]]'

In [8]:
packets = []

for line in puzzle.split('\n'):
    if line != '':
        packets.append(json.loads(line))
        
packets

[[1, 1, 3, 1, 1],
 [1, 1, 5, 1, 1],
 [[1], [2, 3, 4]],
 [[1], 4],
 [9],
 [[8, 7, 6]],
 [[4, 4], 4, 4],
 [[4, 4], 4, 4, 4],
 [7, 7, 7, 7],
 [7, 7, 7],
 [],
 [3],
 [[[]]],
 [[]],
 [1, [2, [3, [4, [5, 6, 7]]]], 8, 9],
 [1, [2, [3, [4, [5, 6, 0]]]], 8, 9],
 [[2]],
 [[6]]]

### Strategy

Let's start simple.

* Compare packet 2 to 1. If out of order, swap.
* Compare 3 to 2. If out of order, swap, and compare 3 to 1. ...
* Compare n to n-1. If out of order, swap, and repeat until packet n in order. 

In [9]:
import copy
# because lists are mutable, and my check_order function pops off elements

In [10]:
n = 1
inplace = False

while n < len(packets):
    p = n
    while p > 0:
        order = check_order(copy.deepcopy(packets[p-1]), copy.deepcopy(packets[p]))
        if order == 'not in order':
            packets[p-1], packets[p] = packets[p], packets[p-1]
            p -= 1
        else:
            p = 0      
    n += 1

In [11]:
packets

[[],
 [[]],
 [[[]]],
 [1, 1, 3, 1, 1],
 [1, 1, 5, 1, 1],
 [[1], [2, 3, 4]],
 [1, [2, [3, [4, [5, 6, 0]]]], 8, 9],
 [1, [2, [3, [4, [5, 6, 7]]]], 8, 9],
 [[1], 4],
 [[2]],
 [3],
 [[4, 4], 4, 4],
 [[4, 4], 4, 4, 4],
 [[6]],
 [7, 7, 7],
 [7, 7, 7, 7],
 [[8, 7, 6]],
 [9]]

In [12]:
(packets.index([[2]]) + 1) * (packets.index([[6]]) + 1)

140